In [1]:

import matplotlib as mpl

from netCDF4 import Dataset
import wrf
import xarray as xr
import numpy as np

import cartopy.crs as crs


from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cartopy.feature as cfe

import pandas as pd


import matplotlib.pyplot as plt



from cartopy.feature import NaturalEarthFeature

import cartopy.io.shapereader as shpr

import cartopy.io.img_tiles as cimgt

from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

from matplotlib.cm import get_cmap
from wrf import getvar, interplevel, to_np, get_basemap, latlon_coords
import imageio

In [2]:
stamen_terrain = cimgt.Stamen('terrain-background')

states = NaturalEarthFeature(category="cultural", scale="10m",
                                 facecolor="none",
                                 name="admin_1_states_provinces_shp")
stt = NaturalEarthFeature(category='cultural', 
    name='admin_0_boundary_lines_land',
    scale='10m',facecolor='none')
stt_prv = states
roads = NaturalEarthFeature(category='cultural',
                                     name='roads',
                                     scale='10m',
                                     facecolor='none')

In [3]:
cm_nubes=mpl.colors.ListedColormap(['white', 'whitesmoke', "lightgrey", 'darkgrey',"grey"])

In [ ]:
def gif_viento_nubes(fichero,dominio):
    ncfile =Dataset(fichero)
    for i in range(0,57):

        nubes = getvar(ncfile, "cloudfrac",timeidx=i*3)[0,:]

        p = getvar(ncfile, "pressure",timeidx=i*3)
        ua = getvar(ncfile, "ua",timeidx=i*3)
        va = getvar(ncfile, "va",timeidx=i*3)

        #Saco la dirección del viento a 500 hPa
        va =interplevel(va, p, 925)

        ua =interplevel(ua, p, 925)



        cart_proj = get_cartopy(nubes)
        lats, lons = latlon_coords(nubes)
        stamen_terrain = cimgt.Stamen('terrain-background')
        plt.figure(figsize=(10,10))

        fig = plt.axes(projection=cart_proj)
        fig.set_title('Nubosidad \n Día {} Hora {}:00:00'.format(int(i*3/24)+11,(i*3-int(i*3/24)*24)), fontsize=16)
        fig.add_feature(roads,linewidth=.4, edgecolor="black")
        plt.contourf(to_np(lons), to_np(lats), to_np(nubes),
                             transform=crs.PlateCarree(),
                             levels = [0.25,0.5,0.75,1],
                             cmap=cm_nubes)

        plt.colorbar(ax=fig, shrink=.98)

        plt.quiver(to_np(lons[::3,::3]), to_np(lats[::3,::3]),to_np(ua[::3, ::3]), to_np(va[::3, ::3]),transform=crs.PlateCarree())


        fig.set_xlim(cartopy_xlim(nubes))
        fig.set_ylim(cartopy_ylim(nubes))
        fig.add_image(stamen_terrain, 8)
        fig.coastlines('50m', linewidth=0.8)

        plt.savefig('figs/d{}/vientos_nubes/viento_nubes_{}.png'.format(dominio,i))
        #Para que no se muestren las imágenes
        plt.close()   
        with imageio.get_writer('figs/d{}/viento_nubes.gif'.format(dominio), mode='I') as writer:
                    for i in range(0,57):
                        for j in range(0,3):
                            filename = 'figs/d{}/vientos_nubes/viento_nubes_{}.png'.format(dominio,i)
                            image = imageio.imread(filename)
                            writer.append_data(image)  

#Para pasar de gif a mp4                        
#ffmpeg -i animated.gif -movflags faststart -pix_fmt yuv420p -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" video.mp4     

## Dominio 1

In [8]:
gif_viento_nubes(fichero="wrfout_d01_2021-05-11_00:00:00",dominio="01")

FileNotFoundError: No such file: '/home/fernan/Build_WRF/WRF-4.1.5/run/Codigos/figs/d01/vientos nubes/viento_nubes_0.png'

## Dominio 2


In [ ]:
gif_viento_nubes(fichero="wrfout_d02_2021-05-11_00:00:00",dominio="02")

## Dominio 3


In [14]:
gif_viento_nubes(fichero="wrfout_d03_2021-05-11_00:00:00",dominio="03")

# Los 3 dominios

In [45]:
# Abrir el dataset
def gif_viento_nubes_3_doms(fichero):
    ncfile_01 = Dataset(fichero[0])
    ncfile_02 = Dataset(fichero[1])
    ncfile_03 = Dataset(fichero[2])
    stamen_terrain = cimgt.Stamen('terrain-background')
    
    for i in range(0,57):
        #Para interpolar y mostrar los vientos
        #dominio 1
        nubes_01 = getvar(ncfile_01, "cloudfrac",timeidx=i*3)[0,:]
        p_01 = getvar(ncfile_01, "pressure",timeidx=i*3)
        ua_01 = getvar(ncfile_01, "ua",timeidx=i*3)
        va_01 = getvar(ncfile_01, "va",timeidx=i*3)
        #Saco la dirección del viento a 500 hPa
        va_01 =interplevel(va_01, p_01, 500)
        ua_01 =interplevel(ua_01, p_01, 500)

        
        
        #dominio 2
        nubes_02 = getvar(ncfile_02, "cloudfrac",timeidx=i*3)[0,:]
        p_02 = getvar(ncfile_02, "pressure",timeidx=i*3)
        ua_02 = getvar(ncfile_02, "ua",timeidx=i*3)
        va_02 = getvar(ncfile_02, "va",timeidx=i*3)
        
        va_02 =interplevel(va_02, p_02, 500)
        ua_02 =interplevel(ua_02, p_02, 500)
        
        #dominio 3
        nubes_03 = getvar(ncfile_03, "cloudfrac",timeidx=i*3)[0,:]
        p_03 = getvar(ncfile_03, "pressure",timeidx=i*3)
        ua_03 = getvar(ncfile_03, "ua",timeidx=i*3)
        va_03 = getvar(ncfile_03, "va",timeidx=i*3)
        
        va_03 =interplevel(va_03, p_03, 500)
        ua_03 =interplevel(ua_03, p_03, 500)
        
        
        
        
        #Para leer la proyección en la que se ha hecho
        cart_proj = get_cartopy(nubes_03)

        # Crear la figura para mostrarlo
        fig = plt.figure(figsize=(15, 6))
    
        
        niveles = [0.25,0.5,0.75,1]
        
        
        
        #Ciudad de Madrid
        # Para leer la latitud y la longitud
        lats, lons = latlon_coords(nubes_03)
        
        ax = fig.add_subplot(1, 5, (1,4), projection=cart_proj)
    
        
        ax.set_title('Nubosidad en Madrid \n Día {} Hora {}:00:00'.format(int(i*3/24)+11,(i*3-int(i*3/24)*24)), fontsize=16)
        
        ax.coastlines('50m', linewidth=0.8)

        # Set the map bounds
        ax.set_xlim(cartopy_xlim(nubes_03))
        ax.set_ylim(cartopy_ylim(nubes_03))
        
        plt.contourf(to_np(lons), to_np(lats), to_np(nubes_03),
                             transform=crs.PlateCarree(),
                             levels = niveles,
                             cmap=cm_nubes)

        plt.colorbar(ax=ax ,  shrink=.98)

        plt.quiver(to_np(lons[::3,::3]), to_np(lats[::3,::3]),to_np(ua_03[::3, ::3]), to_np(va_03[::3, ::3]),transform=crs.PlateCarree())


        ax.add_image(stamen_terrain, 8)
        ax.coastlines('50m', linewidth=0.8)
        
        ax.add_feature(stt, linewidth=0.2, edgecolor='black')
        
        #Mostrar las carreteras 
        ax.add_feature(roads,linewidth=.4, edgecolor="black")

        # Add the gridlines
        ax.gridlines(color="gray", linestyle="dotted")

        
        
        
        
        
        # Comunidad de Madrid
        lats, lons = latlon_coords(nubes_02)
        
        ax_2 = fig.add_subplot(2, 5, 5,projection=cart_proj)
        
        ax_2.set_xlim(cartopy_xlim(nubes_02))
        ax_2.set_ylim(cartopy_ylim(nubes_02))
        
        ax_2.set_title('Comunidad de Madrid', fontsize=8)
        
        ax_2.coastlines('50m', linewidth=0.8)
        
       
      
        plt.contourf(to_np(lons), to_np(lats), to_np(nubes_02),
                             transform=crs.PlateCarree(),
                             levels = niveles,
                             cmap=cm_nubes)

        plt.colorbar(ax=ax_2, shrink=.98)

        plt.quiver(to_np(lons[::3,::3]), to_np(lats[::3,::3]),to_np(ua_02[::3, ::3]),
                   to_np(va_02[::3, ::3]),transform=crs.PlateCarree())


        ax_2.add_image(stamen_terrain, 8)
        
        
        ax_2.add_feature(stt, linewidth=0.2, edgecolor='black')
        
        #Mostrar las carreteras 
        ax_2.add_feature(roads,linewidth=.4, edgecolor="black")
        ax_2.add_feature(stt_prv, linewidth=0.2, edgecolor='black')
        
        # Add the gridlines
        ax_2.gridlines(color="gray", linestyle="dotted")
        
        
        
        
        
        #España
        lats, lons = latlon_coords(nubes_01)
        
        ax_3 = fig.add_subplot(2, 5, 10,projection=cart_proj)
        
        ax_3.set_xlim(cartopy_xlim(nubes_01))
        ax_3.set_ylim(cartopy_ylim(nubes_01))
        
        ax_3.set_title('España', fontsize=8)
        
        ax_3.coastlines('50m', linewidth=0.8)

        plt.contourf(to_np(lons), to_np(lats), to_np(nubes_01),
                             transform=crs.PlateCarree(),
                             levels = niveles,cmap=cm_nubes)
                             

        

        plt.quiver(to_np(lons[::3,::3]), to_np(lats[::3,::3]),
                   to_np(ua_01[::3, ::3]), to_np(va_01[::3, ::3]),
                   transform=crs.PlateCarree())


        ax_3.add_image(stamen_terrain, 8)
        ax_3.coastlines('50m', linewidth=0.8)
        
        ax_3.add_feature(stt, linewidth=0.2, edgecolor='black')
        
        #Mostrar las carreteras 
        ax_3.add_feature(roads,linewidth=.4, edgecolor="black")
        ax_3.add_feature(stt_prv, linewidth=0.2, edgecolor='black')
        
        # Add the gridlines
        ax_3.gridlines(color="gray", linestyle="dotted")

        
        
        
        plt.savefig('figs/viento_nubes/viento_nubes{}.png'.format(i))
        #Para que no se muestren las imágenes
        plt.close(fig)
        
        
        

    with imageio.get_writer('figs/viento_nubes.gif', mode='I') as writer:
            for i in range(0,57):
                for j in range(0,3):
                    filename = 'figs/viento_nubes/viento_nubes{}.png'.format(i)
                    image = imageio.imread(filename)
                    writer.append_data(image)    

In [46]:
gif_viento_nubes_3_doms(['wrfout_d01_2021-05-11_00:00:00',
                        'wrfout_d02_2021-05-11_00:00:00',
                        'wrfout_d03_2021-05-11_00:00:00'])